In [2]:
import pandas as pd
import math
import numpy as np

In [3]:
report = pd.read_csv('https://raw.githubusercontent.com/HiddenS1/Marshall-Project-on-FBI-s-Crime-Data-Quality/main/data/Index_Crimes_by_County_and_Agency__Beginning_1990%20(1).csv')
NY_part = pd.read_csv("https://raw.githubusercontent.com/HiddenS1/Marshall-Project-on-FBI-s-Crime-Data-Quality/main/data/NY_part.csv")

In [62]:
# In order to merge the two tables on agency names
# we categorize them by agency type
city_words = ['city pd', 'town pd', 'vg pd']
county_word = ['county sheriff', 'county']
sp_word = 'state police'

In [63]:
is_city = [any(word in agency.lower() for word in city_words) for agency in report.Agency]
city = report.loc[is_city]

In [64]:
city_NY = NY_part.loc[NY_part.agency_type=='City','agency_name_full'].unique().tolist()

In [65]:
city_report = city.Agency.unique().tolist()

In [8]:
from thefuzz import fuzz, process

In [66]:
city_connect = {}
for i in range(len(city_NY)):
    best_match = process.extractOne(city_NY[i], city_report, scorer=fuzz.ratio)
    city_connect.update({city_NY[i]: best_match[0]})

In [67]:
city_connect['lakewood-busti police department'] = 'Lakewood-Busti PD'
city_connect['east aurora village pd east aurora'] = 'East Aurora-Town Of Aurora PD'
city_connect['port washington police district'] = 'Port Washington Police District'
city_connect['troy police department'] = 'Troy City PD'
city_connect['south nyack - grand view village police department'] = 'South Nyack-Grand View PD'
city_connect.pop('columbia greene humane society')
city_connect.pop('theresa village pd theresa')
city_connect.pop('new york city police department')
city_connect.pop('barker village police department')

'Waterville Vg PD'

In [68]:
city_connect.pop('galway vlg police dept')
city_connect.pop('south nyack - grand view village police department')
city_connect.pop('port washington police district')
city_connect.pop('east aurora village pd east aurora')
city_connect['endicott village police department'] = 'Endicott Vg PD'
city_connect['webster town police department'] = 'Webster Town and Vg PD'

In [69]:
city_connect.pop('lakewood-busti police department')

'Lakewood-Busti PD'

In [70]:
county_NY = NY_part.loc[NY_part.agency_type=='County','agency_name_full'].unique().tolist()

In [71]:
is_county = [any(word in agency.lower() for word in county_word) for agency in report.Agency]
county = report.loc[is_county]
county_report = county.Agency.unique().tolist()

In [72]:
county_connect = {}
for i in range(len(county_NY)):
    best_match = process.extractOne(county_NY[i], county_report, scorer=fuzz.ratio)
    county_connect.update({county_NY[i]: best_match[0]})

In [73]:
county_connect['nassau county police department'] = 'Nassau County PD'
county_connect['suffolk county police department'] = 'Suffolk County PD'
county_connect['westchester county department of public safety'] = 'Westchester Co Dept Pub Safety'

In [74]:
city_connect.update(county_connect)

In [49]:
# Ignore stopwords and highlight keywords
stopwords = ['of', 'and', 'in', 'police', 'department', 'pd', 'dept']
keywords = ['vlg', 'village', 'vg', 'county', 'sheriff']

In [77]:
def custom_scorer(s1, s2, alpha, beta):
    s1 = s1.replace('-', ' ')
    s2 = s2.replace('-', ' ')
    words1 = [i for i in s1.lower().split() if i not in stopwords]
    words2 = [i for i in s2.lower().split() if i not in stopwords]
    is_key1 = any([i in keywords for i in words1])
    is_key2 = any([i in keywords for i in words2])
    similarity = fuzz.ratio(' '.join(words1), ' '.join(words2))  
    if is_key1 and is_key2:
        similarity += alpha
    first_word_similarity = fuzz.ratio(' '.join(words1[:1]), ' '.join(words2[:1]))
    return similarity + first_word_similarity * beta

In [79]:
def conn(c1, c2, alpha, beta):
    city = {}
    for item1 in c1:
        highest_similarity = 0
        matching_item = None
        for item2 in c2:
            similarity = custom_scorer(item1, item2, alpha, beta)
            if similarity > highest_similarity:
                highest_similarity = similarity
                matching_item = item2
        city[item1] = matching_item
    return(city)

In [52]:
def dist(c1, c2, alpha, beta):
    city = {}
    for item1 in c1:
        highest_similarity = 0
        matching_item = None
        for item2 in c2:
            similarity = custom_scorer(item1, item2, alpha, beta)
            if similarity > highest_similarity:
                highest_similarity = similarity
                matching_item = item2
        city[item1] = matching_item
    sum = 0
    for item in city_connect:
        if city_connect[item] != city[item]:
            print(item, city_connect[item], city[item])
            sum += 1
    return(sum)

In [75]:
NY_lst = city_NY + county_NY
report_lst = city_report + county_report

In [76]:
a_lst = [5, 10, 20, 30]
b_lst = [0.1, 0.5, 1, 1.5, 2]
for a in a_lst:
    for b in b_lst:
        print(dist(NY_lst, report_lst, a, b))

westchester county department of public safety Westchester Co Dept Pub Safety Westchester County State Police
1
town of pine plains police dept Pine Plains Town PD Stony Point Town PD
westchester county department of public safety Westchester Co Dept Pub Safety Westchester County State Police
2
town of pine plains police dept Pine Plains Town PD Stony Point Town PD
st. lawrence county sheriff's office St Lawrence County Sheriff St. Lawrence County (RR-CSX)
westchester county department of public safety Westchester Co Dept Pub Safety Westchester County State Police
3
town of pine plains police dept Pine Plains Town PD Stony Point Town PD
st. lawrence county sheriff's office St Lawrence County Sheriff St. Lawrence County (RR-CSX)
westchester county department of public safety Westchester Co Dept Pub Safety Westchester County State Police
3
town of pine plains police dept Pine Plains Town PD Stony Point Town PD
le roy village police department LeRoy Vg PD Glen Park Vg PD
st. lawrence coun

In [80]:
allrp = report.Agency.unique().tolist()
allny = NY_part.agency_name_full.unique().tolist()

In [81]:
city = conn(allny, allrp, 5, 0.1)

In [82]:
city

{'albany police department': 'Albany City PD',
 "albany county sheriff's office": 'Albany County Sheriff',
 'cohoes police department': 'Cohoes City PD',
 'watervliet police department': 'Watervliet City PD',
 'green island vlg police dept': 'Green Island Vg PD',
 'altamont village police department': 'Altamont Vg PD',
 'menands village police department': 'Menands Vg PD',
 'bethlehem town police department': 'Bethlehem Town PD',
 'guilderland town police department': 'Guilderland Town PD',
 'colonie town police department': 'Colonie Town PD',
 'nys parks and recreation office albany': 'SUNY Police - Albany',
 'coeymans town police department': 'Coeymans Town PD',
 "allegany county sheriff's office": 'Allegany County Sheriff',
 'alfred village police department': 'Alfred Vg PD',
 'andover village police department': 'Andover Vg PD',
 'angelica vlg police dept': 'Angelica Vg PD',
 'bolivar village police department': 'Bolivar Vg PD',
 'cuba town police department': 'Cuba Town PD',
 'wel